# Notebook: Convert Model Results to Latex

This notebook is used to load the .json files with the model performance in order to convert them into a latex table for the paper

In [87]:
# Todo: Schauen, ob es die Metriken auch bei anderen Modellen gibt
# Todo: 1.000 <- Punkte einfügen
# Todo: Soll bei nur Real gehen
# Todo: Soll bei allen Tasks gehen
# Todo: bei f1 micro etc 3 nachkommastellen
# Schauen, dass es bei jedem Task "eval_f1_micro", "eval_f1_macro", "accuracy"

## Packages

In [88]:
import json

## Settings / Constants

In [89]:
RESULTS_PATH_BASE = "../07 train classifier/results_json/results_"
LLMS = ["Llama70B", "GPT-3"]
LLM_PAPER_TITLE = {"Llama70B": "Llama2-70B", "GPT-3": "GPT-3.5-turbo"}
# , "aspect_category_sentiment", "end_2_end_absa" ,"target_aspect_sentiment_detection"]
ABSA_TASKS = ["aspect_category"]
SYNTH_COMBINATIONS = {
    "random": [
        {"real": 500, "synth": 500},
        {"real": 500, "synth": 1000},
        {"real": 500, "synth": 1500}
    ], "fixed": [
        {"real": 25, "synth": 475},
        {"real": 25, "synth": 975},
        {"real": 25, "synth": 1975}
    ]}

In [90]:
FEW_SHOT_STRATEGY = {25: "25 fixed examples", 500: "25 random examples"}

## Code

In [91]:
def format_thousands(number):
    formatted_number = "{:,}".format(number)
    return formatted_number

def round_to_three_decimals(number):
    rounded_number = round(number, 3)
    return rounded_number

### Report Main Metrics

In [92]:
for absa_task in ABSA_TASKS:
    print(absa_task, ":\n")
    for n_real in [500, 1000, 2000]:
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"
        with open(json_path, 'r') as json_file:
            results = json.load(json_file)
        print(
            f"- & - & {format_thousands(n_real)} & 0 & {round_to_three_decimals(results['eval_f1_micro'])} & {round_to_three_decimals(results['eval_f1_macro'])} & {round_to_three_decimals(results['eval_accuracy'])} \\\\")
    print("\\hline")
    for few_shot_condition in SYNTH_COMBINATIONS.keys():
        for llm in LLMS:
            for freq in SYNTH_COMBINATIONS[few_shot_condition]:
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)
                # print(f"results: {absa_task}, {llm}, {few_shot_condition}, n_real: {n_real}, n_synth: {n_synth}", results)
                if absa_task == "TASD":
                    f1_metrics = f"{round_to_three_decimals(results['eval_f1_micro'])} & {round_to_three_decimals(results['eval_f1_macro'])}"
                else:
                    f1_metrics = f"{round_to_three_decimals(results['eval_f1_micro'])}"
                print(
                    f"{LLM_PAPER_TITLE[llm]} & {FEW_SHOT_STRATEGY[n_real]} & {format_thousands(n_real)} & {format_thousands(n_synth)} & {f1_metrics} & {round_to_three_decimals(results['eval_accuracy'])} & {round_to_three_decimals(results['eval_accuracy'])} \\\\")
        print("\hline")

aspect_category :

- & - & 500 & 0 & 0.061 & 0.033 & 0.026 \\
- & - & 1,000 & 0 & 0.061 & 0.033 & 0.026 \\
- & - & 2,000 & 0 & 0.061 & 0.033 & 0.026 \\
\hline
Llama2-70B & 25 random examples & 500 & 500 & 0.061 & 0.026 & 0.026 \\
Llama2-70B & 25 random examples & 500 & 1,000 & 0.061 & 0.026 & 0.026 \\
Llama2-70B & 25 random examples & 500 & 1,500 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 random examples & 500 & 500 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 random examples & 500 & 1,000 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 random examples & 500 & 1,500 & 0.061 & 0.026 & 0.026 \\
\hline
Llama2-70B & 25 fixed examples & 25 & 475 & 0.061 & 0.026 & 0.026 \\
Llama2-70B & 25 fixed examples & 25 & 975 & 0.061 & 0.026 & 0.026 \\
Llama2-70B & 25 fixed examples & 25 & 1,975 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 fixed examples & 25 & 475 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 fixed examples & 25 & 975 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 fixed examples & 25

### Report Metrics Fine-Grained Metrics

#### ACD

In [96]:
absa_task = "aspect_category"
for n_real in [500, 1000, 2000]:
    json_path = RESULTS_PATH_BASE + \
        f"only_real_real{n_real}_synth0_{absa_task}_random.json"
    with open(json_path, 'r') as json_file:
        results = json.load(json_file)
    class_wise_metrics = ""
    for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
       for metric in ["f1", "accuracy"]:
           class_wise_metrics += f"{round_to_three_decimals(results[f'eval_{metric}_{ac}'])} &"
    print(f"- & - & {format_thousands(n_real)} & 0 & {class_wise_metrics[:-1]} \\\\")
print("\\hline")
for few_shot_condition in SYNTH_COMBINATIONS.keys():
    for llm in LLMS:
        for freq in SYNTH_COMBINATIONS[few_shot_condition]:
            n_real = freq["real"]
            n_synth = freq["synth"]
            json_path = RESULTS_PATH_BASE + llm + \
                f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
            with open(json_path, 'r') as json_file:
                results = json.load(json_file)
            # print(f"results: {absa_task}, {llm}, {few_shot_condition}, n_real: {n_real}, n_synth: {n_synth}", results)
            if absa_task == "TASD":
                f1_metrics = f"{round_to_three_decimals(results['eval_f1_micro'])} & {round_to_three_decimals(results['eval_f1_macro'])}"
            else:
                f1_metrics = f"{round_to_three_decimals(results['eval_f1_micro'])}"

            class_wise_metrics = ""
            for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
                for metric in ["f1", "accuracy"]:
                   class_wise_metrics += f"{round_to_three_decimals(results[f'eval_{metric}_{ac}'])} &"

            print(
                    f"{LLM_PAPER_TITLE[llm]} & {FEW_SHOT_STRATEGY[n_real]} & {format_thousands(n_real)} & {format_thousands(n_synth)} & {class_wise_metrics[:-1]} \\\\")
    print("\hline")

- & - & 500 & 0 & 0.0 &0.736 &0.165 &0.614 &0.0 &0.686 &0.0 &0.872 &0.0 &0.926  \\
- & - & 1,000 & 0 & 0.0 &0.736 &0.165 &0.614 &0.0 &0.686 &0.0 &0.872 &0.0 &0.926  \\
- & - & 2,000 & 0 & 0.0 &0.736 &0.165 &0.614 &0.0 &0.686 &0.0 &0.872 &0.0 &0.926  \\
\hline
Llama2-70B & 25 random examples & 500 & 500 & 0.0 &0.736 &0.165 &0.614 &0.0 &0.686 &0.0 &0.872 &0.0 &0.926  \\
Llama2-70B & 25 random examples & 500 & 1,000 & 0.0 &0.736 &0.165 &0.614 &0.0 &0.686 &0.0 &0.872 &0.0 &0.926  \\
Llama2-70B & 25 random examples & 500 & 1,500 & 0.0 &0.736 &0.165 &0.614 &0.0 &0.686 &0.0 &0.872 &0.0 &0.926  \\
GPT-3.5-turbo & 25 random examples & 500 & 500 & 0.0 &0.736 &0.165 &0.614 &0.0 &0.686 &0.0 &0.872 &0.0 &0.926  \\
GPT-3.5-turbo & 25 random examples & 500 & 1,000 & 0.0 &0.736 &0.165 &0.614 &0.0 &0.686 &0.0 &0.872 &0.0 &0.926  \\
GPT-3.5-turbo & 25 random examples & 500 & 1,500 & 0.0 &0.736 &0.165 &0.614 &0.0 &0.686 &0.0 &0.872 &0.0 &0.926  \\
\hline
Llama2-70B & 25 fixed examples & 25 & 475 & 0.0 &0

#### ACSA

In [ ]:
absa_task = "aspect_category_sentiment"

idx = 0
for aspect_categories in [["GENERAL-IMPRESSION", "FOOD"], ["SERVICE", "AMBIENCE", "PRICE"]]:
    print("Table for:", aspect_categories, "\n\n")
    for n_real in [500, 1000, 2000]:
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"
        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        condition_string = f"- & - & {format_thousands(n_real)} & 0 &"
        metrics_class_wise = ""
        for ac in aspect_categories:
            for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                for metric in ["eval_f1", "eval_precision", "eval_recall"]:
                    metrics_class_wise += f" {round_to_three_decimals(results[f'{metric}_{ac}-{polarity}'])} &"

        print(condition_string + metrics_class_wise + "\\\\") if idx == 0 else print(metrics_class_wise[:-1] + "\\\\")

    print("\\hline")
    for few_shot_condition in SYNTH_COMBINATIONS.keys():
        for llm in LLMS:
            for freq in SYNTH_COMBINATIONS[few_shot_condition]:
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                condition_string = f"{LLM_PAPER_TITLE[llm]} & {FEW_SHOT_STRATEGY[n_real]} & {format_thousands(n_real)} & {format_thousands(n_synth)} &"
                metrics_class_wise = ""
                for ac in aspect_categories:
                    for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                        for metric in ["eval_f1", "eval_precision", "eval_recall"]:
                            metrics_class_wise += f" {round_to_three_decimals(results[f'{metric}_{ac}-{polarity}'])} &"

                print(condition_string + metrics_class_wise + "\\\\") if idx == 0 else print(metrics_class_wise[:-1] + "\\\\")
        print("\hline")
    idx += 1

Table for: ['GENERAL-IMPRESSION', 'FOOD'] 


- & - & 500 & 0 & 0.278 & 0.234 & 0.342 & 0.044 & 0.026 & 0.143 & 0.0 & 0.0 & 0.0 & 0.152 & 0.241 & 0.111 & 0.079 & 0.044 & 0.348 & 0.381 & 0.268 & 0.662 &\\
- & - & 1,000 & 0 & 0.278 & 0.234 & 0.342 & 0.044 & 0.026 & 0.143 & 0.0 & 0.0 & 0.0 & 0.152 & 0.241 & 0.111 & 0.079 & 0.044 & 0.348 & 0.381 & 0.268 & 0.662 &\\
- & - & 2,000 & 0 & 0.278 & 0.234 & 0.342 & 0.044 & 0.026 & 0.143 & 0.0 & 0.0 & 0.0 & 0.152 & 0.241 & 0.111 & 0.079 & 0.044 & 0.348 & 0.381 & 0.268 & 0.662 &\\
\hline
Llama2-70B & 25 random examples & 500 & 500 & 0.278 & 0.234 & 0.342 & 0.044 & 0.026 & 0.143 & 0.0 & 0.0 & 0.0 & 0.152 & 0.241 & 0.111 & 0.079 & 0.044 & 0.348 & 0.381 & 0.268 & 0.662 &\\
Llama2-70B & 25 random examples & 500 & 1,000 & 0.278 & 0.234 & 0.342 & 0.044 & 0.026 & 0.143 & 0.0 & 0.0 & 0.0 & 0.152 & 0.241 & 0.111 & 0.079 & 0.044 & 0.348 & 0.381 & 0.268 & 0.662 &\\
Llama2-70B & 25 random examples & 500 & 1,500 & 0.278 & 0.234 & 0.342 & 0.044 & 0.026 & 0.143 & 

In [ ]:
results

{'LLM_NAME': 'only_real',
 'N_REAL': 500,
 'N_SYNTH': 0,
 'TARGET': 'aspect_category_sentiment',
 'LLM_SAMPLING': 'random',
 'eval_loss': 0.6316537976264953,
 'eval_accuracy': 0.0,
 'eval_hamming_loss': 0.26893333333333336,
 'eval_f1_macro': 0.11427978789259886,
 'eval_f1_micro': 0.15853149770546515,
 'eval_f1_weighted': 0.1873516502346942,
 'eval_precision_GENERAL-IMPRESSION': 0.18120805369127516,
 'eval_recall_GENERAL-IMPRESSION': 0.20454545454545456,
 'eval_f1_GENERAL-IMPRESSION': 0.19217081850533807,
 'eval_accuracy_GENERAL-IMPRESSION': 0.648,
 'eval_precision_FOOD': 0.1712962962962963,
 'eval_recall_FOOD': 0.33636363636363636,
 'eval_f1_FOOD': 0.22699386503067484,
 'eval_accuracy_FOOD': 0.252,
 'eval_precision_SERVICE': 0.12412177985948478,
 'eval_recall_SERVICE': 0.3375796178343949,
 'eval_f1_SERVICE': 0.18150684931506852,
 'eval_accuracy_SERVICE': 0.33,
 'eval_precision_AMBIENCE': 0.06292134831460675,
 'eval_recall_AMBIENCE': 0.4307692307692308,
 'eval_f1_AMBIENCE': 0.1098039215